# Timeseries forecasting for weather prediction

This lab is based on [this](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) Keras tutorial. Our goal is to perform timeseries forecasting using a Recurrent Neural Network based on the [Long Short-Term Memory (LSTM)  block](https://builtin.com/data-science/recurrent-neural-networks-and-lstm).

Your task is simple: See if you can figure out how this works! If you succeed, you should have a pretty good understanding of how RNNs work.

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import keras

## Climate Data Time-Series

We will be using Jena Climate dataset recorded by the
[Max Planck Institute for Biogeochemistry](https://www.bgc-jena.mpg.de/wetter/).
The dataset consists of 14 features such as temperature, pressure, humidity etc, recorded once per 10 minutes using a weather station.

The table below shows the column names, their value formats, and their description.

Index| Features      |Format             |Description
-----|---------------|-------------------|-----------------------
1    |Date Time      |01.01.2009 00:10:00|Date-time reference
2    |p (mbar)       |996.52             |The pascal SI derived unit of pressure used to quantify internal pressure. Meteorological reports typically state atmospheric pressure in millibars.
3    |T (degC)       |-8.02              |Temperature in Celsius
4    |Tpot (K)       |265.4              |Temperature in Kelvin
5    |Tdew (degC)    |-8.9               |Temperature in Celsius relative to humidity. Dew Point is a measure of the absolute amount of water in the air, the DP is the temperature at which the air cannot hold all the moisture in it and water condenses.
6    |rh (%)         |93.3               |Relative Humidity is a measure of how saturated the air is with water vapor, the %RH determines the amount of water contained within collection objects.
7    |VPmax (mbar)   |3.33               |Saturation vapor pressure
8    |VPact (mbar)   |3.11               |Vapor pressure
9    |VPdef (mbar)   |0.22               |Vapor pressure deficit
10   |sh (g/kg)      |1.94               |Specific humidity
11   |H2OC (mmol/mol)|3.12               |Water vapor concentration
12   |rho (g/m ** 3) |1307.75            |Airtight
13   |wv (m/s)       |1.03               |Wind speed
14   |max. wv (m/s)  |1.75               |Maximum wind speed
15   |wd (deg)       |152.3              |Wind direction in degrees

Download dataset and load it into a [Pandas dataframe](https://www.datacamp.com/tutorial/pandas-tutorial-dataframe-python).

In [ ]:
from zipfile import ZipFile

uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"
df = pd.read_csv(csv_path)

## Raw Data Visualization

To give us a sense of the data we are working with, each feature has been plotted below.
This shows the distinct pattern of each feature over the time period from 2009 to 2016.
Perhaps you will notice that there seems to be some anomalies or outliers in the data. These will be ignored here, but in general they should addressed, for instance through [data cleaning](https://en.wikipedia.org/wiki/Data_cleansing).

In [ ]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"

def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=7, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()


show_raw_visualization(df)


## Goal
Our goal is to train a model that can predict the temperature 12 hours into the future, given weather station measurements from the past 120 hours (5 days).

In the context of what was covered in Lecture 11, this mostly resembles **Sequence classification**, where the goal is to predict a label given a sequence of inputs. The only difference here is that we will be doing regression instead of classification. So we could call it **sequence regression**, where the goal is to predict a real-valued number (the temperature) given a sequence of inputs (past measurements from the weather station).

## Data Preprocessing

Before we can build and train our model, we will need to do some preprocessing of our data.

### Feature selection
As stated above, there are a total of 14 features but as some of them are strongly correlated (for instance, Relative Humidity and Specific Humidity), we will only be using a subset of them. Specifically, we will be using the following 7:

*Pressure, Temperature, Saturation vapor pressure, Vapor pressure deficit, Specific humidity, Airtight, Wind speed*.

In [ ]:
# Feature selection
print(
    "The selected features are:",
    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
)
selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
features = df[selected_features]
features.index = df[date_time_key] # Make timestamp the index (this is Pandas-stuff)

### Feature normalization
Since every feature has values with
varying ranges, we do normalization to confine feature values to a range of `[0, 1]` before training our RNN. We do this by subtracting the mean and dividing by the standard deviation of each feature.

In [ ]:
# Utility function to perform data normalization
def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    return (data - data_mean) / data_std

# Feature normalization
features = normalize(features.values)
features = pd.DataFrame(features)

### Train / validation split
There are approximately 450.000 observations, of which we will be using initial 71.5 % for training and the remaining 28.5 % for validation.

In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))

# Split inputs (x) into train and validation
train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

### Downsampling and creating input / output pairs
Observations are recorded every 10 minutes (6 times per hour). We will resample one point per hour since no drastic change is expected within 60 minutes.

To train and test the model we need to create input / output pairs. The input is a sequence $[x_1, x_2, ..., x_{120}]$ of 120 past measurements from the weather station (5 days = 120 hours). The output (or the target) $y$ is the temperate 12 timestamps (12 hours) into the future, relative to the timestamp of the last measurement $x_{120}$.

Let's do both of these steps for the training data:


In [ ]:
# Downsample
step = 6
train_data = train_data.values[::step]

x_train = train_data

# The outputs (y) start from the 132nd observation (120 + 12)
past = 120  # Number of samples in input (x)
future = 12 # Number of samples to look into the future to get the output (y)
start = past + future
y_train = train_data[start:,1] # 1 is the index of the Temperature that we want to predict.

print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)

The [`timeseries_dataset_from_array`](https://keras.io/api/data_loading/timeseries/) function takes in a sequence of datapoints gathered at equal intervals, along with time series parameters such as length of the sequences/windows, spacing between two sequence/windows, etc., to produce batches of sub-timeseries inputs ($[x_1, x_2, ..., x_{120}]$) and targets ($y$) sampled from the main timeseries.

In [ ]:
batch_size = 256
sequence_length = 120
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    batch_size=batch_size,
)

# Print shapes (256 is the batch size)
for batch in dataset_train.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Let's build the validation set the same way:

In [ ]:
# Downsample
val_data = val_data.values[::step]
x_val = val_data
y_val = val_data[start:,1] # Again 1 is the index of the temperature
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    batch_size=batch_size,
)

# Print shapes (256 is the batch size)
for batch in dataset_val.take(1):
    inputs, targets = batch
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

## Model
Let's build a simple RNN. To address vanishing gradients we will be using the LSTM block.

Verify for yourself that the input $[x_1, x_2, ..., x_{120}]$ has shape $120\times7$. The output ($y$) is just is scalar, representing the temperature 12 timesteps into the future as explained above.

**Note:** The model might initially look like any fully connected neural network, but recall what is going on under the hood. The LSTM processes the input sequence one token at a time and outputs a hidden state for every token:

$h_t = f_{LSTM}(h_{t-1}, x_t)$

The last hidden state $h_{120}$ then goes into a dense layer (a fully connected NN) that predicts the temperature.

$y_{predicted} = f_{DENSE}(h_{120})$

In [ ]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

## Training
(This will take a few minutes...).

Note that since we are doing regression, we are using the mean squared error loss.

We'll use EarlyStopping callback to interrupt training when the validation loss is not longer improving.

In [ ]:
learning_rate = 0.001
epochs = 10

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback],
)

We can visualize the loss with the function below. After one point, the loss stops
decreasing.

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualize_loss(history, "Training and Validation Loss")

## Prediction

The trained model above is now able to make predictions for 5 sets of values from the
validation set.

The plots below show the temperature 120 timestamps (hours) into the past from the point of reference, and then it shows the predicted temperature and the true temperature 12 timestamps (hours) into the future.

**Note:** That because of the normalization the predictions are not the actual temperatures. I will leave it as an exercise for you to de-normalize the predictions.

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return


for x, y in dataset_val.take(5):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), model.predict(x)[0]],
        12,
        "Single Step Prediction",
    )